In [33]:
#Google Drive Bağlantisi
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [24]:
%cd gdrive/My Drive/Colab Notebooks/Derin Ogrenme

[Errno 2] No such file or directory: 'gdrive/My Drive/Colab Notebooks/Derin Ogrenme'
/content/gdrive/My Drive/Colab Notebooks/Derin Ogrenme


In [25]:
%ls

basarili_model.h5  GözAnaliz.ipynb  kahverengiGoz.jpg  yesilGoz.jpg
data/              insan.jpg        maviGoz.jpg


In [26]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
img_width, img_height = 110,110

train_data_yolu = 'data/train'
validation_data_yolu = 'data/validation'
train_ornek_sayisi = 1862
validation_ornek_sayisi = 600
epochs = 20
batch_size = 1  #Her seferinde egitim icin alinacak veri miktari
#train_ornek_sayisi, validation_ornek_sayisi ile batch_size tam bolunmeli

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [27]:
model = Sequential()
#Giris katmani ve Conv1
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#Conv2
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#Conv3
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#Flatten Duzlestirme
model.add(Flatten())
#Dense= FC (Fully Connected Layer Olusturuluyor
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5)) #Dropout Factorization
model.add(Dense(3))
#Sinif sayisi
#Binary Siniflama icin :model.add(Dense(1))
model.add(Activation('sigmoid'))

#Model Derleme
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [28]:
#Train islemi icin kullanilan verileri cogaltiyor
train_datalar = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#datalar = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
#        shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

#Test islemi icin goruntuleri rescale boyutlandirma yapiyor
test_datalar = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datalar.flow_from_directory(
    train_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
#Binary classification (ikili siniflama) : class_mode='binary'
#Multiclass (Coklu sinif) : class_mode='categorical'

validation_generator = test_datalar.flow_from_directory(
    validation_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
#Binary classification (ikili siniflama) : class_mode='binary'
#Multiclass (Coklu sinif) : class_mode='categorical'

Found 1863 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [ ]:
#Modelin Eğitimi
model.fit(
    train_generator,
    steps_per_epoch=train_ornek_sayisi // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_ornek_sayisi // batch_size
)

Epoch 1/20
1862/1862 [==============================] - 650s 348ms/step - loss: 0.5613 - accuracy: 0.5854 - val_loss: 0.3515 - val_accuracy: 0.7517
Epoch 2/20
1862/1862 [==============================] - 38s 20ms/step - loss: 0.4511 - accuracy: 0.7513 - val_loss: 0.2354 - val_accuracy: 0.8400
Epoch 3/20
1862/1862 [==============================] - 38s 20ms/step - loss: 0.4507 - accuracy: 0.7642 - val_loss: 0.2598 - val_accuracy: 0.8467
Epoch 4/20
1862/1862 [==============================] - 38s 20ms/step - loss: 0.4552 - accuracy: 0.7889 - val_loss: 0.2393 - val_accuracy: 0.8567
Epoch 5/20
1862/1862 [==============================] - 38s 20ms/step - loss: 0.4575 - accuracy: 0.7803 - val_loss: 0.2474 - val_accuracy: 0.8733
Epoch 6/20
1862/1862 [==============================] - 38s 21ms/step - loss: 0.4673 - accuracy: 0.7836 - val_loss: 0.2497 - val_accuracy: 0.8783
Epoch 7/20
1862/1862 [==============================] - 38s 21ms/step - loss: 0.4261 - accuracy: 0.7986 - val_loss: 0.2308

In [ ]:
model.save_weights('basarili_model.h5')

In [35]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np

model.load_weights('basarili_model.h5')

img = load_img('insan3.jfif', grayscale=False, color_mode='rgb',target_size=(img_width, img_height))

#img = load_img('deneme.jpg',False, (img_width, img_height))
x= img_to_array(img)    #Numpy kutuphanesi ilebir diziye donusturuldu: shape(3, 244, 244)
prediction = model.predict(x.reshape((1, img_width, img_height,3)), batch_size=32, verbose=0) #(1, 244, 244, 3)
print(prediction)

FileNotFoundError: ignored